In [ ]:
# Creación del vector de fechas
fechas = [
    "{:4d}{:02d}".format(year, month)
    for year in range(1995, 2022)
    for month in range(1, 13)
]
fechas = [fecha for fecha in fechas if fecha >= "199507" and fecha <= "202104"]
fechas[0], fechas[-1], len(fechas)

In [ ]:
# Lectura de los archivos
import pandas as pd

url = (
    "https://raw.githubusercontent.com/"
    "jdvelasq/datalabs/master/datasets/precio_bolsa_nacional/csv/{}.csv"
)

org_df = pd.concat([pd.read_csv(url.format(fecha)) for fecha in fechas])
org_df.head()

In [ ]:
# Manipulacion del dataframe
melted_df = pd.melt(
    org_df,
    id_vars="Fecha",
    var_name="Hora",
    value_name="Precio",
)
melted_df = melted_df.sort_values(["Fecha", "Hora"]).reset_index(drop=True)
melted_df.head()

In [ ]:
# Verificación de los tipos para buscar problemas
melted_df.dtypes

In [ ]:
# Registros con precios NA
melted_df.Precio.isna().sum()

In [ ]:
# Tamaño original del dataframe
melted_df.shape

In [ ]:
# Eliminación de NA y tamaño final del dataframe
melted_df = melted_df.dropna()
melted_df.shape

In [ ]:
# Búsqueda de registros con str en los precios
melted_df[melted_df["Precio"].map(lambda x: isinstance(x, str))].sort_values("Precio")

In [ ]:
# Se elimina el '.' de los miles
melted_df["Precio"] = melted_df["Precio"].map(
    lambda x: x.replace(".", "") if isinstance(x, str) else x
)

In [ ]:
# Se reemplaza la ',' por '.'
melted_df["Precio"] = melted_df["Precio"].map(
    lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x
)

In [ ]:
# Vefificación
melted_df["Precio"][melted_df["Precio"].map(lambda x: isinstance(x, str))].sort_values()

In [ ]:
# Revisión de los tipos de datos
melted_df.dtypes

In [ ]:
# Cálculo de los precios diarios y renombramiento de la columna
precios_df = melted_df.groupby(["Fecha"]).agg({"Precio": "mean"})
precios_df = precios_df.rename(columns={"Precio": "Precio diario"})
precios_df.head()

In [ ]:
# Tranformación del indice a tipo fecha
precios_df.index = pd.to_datetime(precios_df.index)

In [ ]:
# Cálculo del precio promedio por mes
precios_df["Precio mensual"] = precios_df.groupby(
    [precios_df.index.year, precios_df.index.month]
)["Precio diario"].transform("mean")

precios_df.head()

In [ ]:
import os
import matplotlib.pyplot as plt

precios_df.plot(style=["gray", "k"], figsize=(8, 3))
plt.xticks(rotation="vertical", fontsize=8)
plt.yticks(fontsize=8)
plt.gca().spines["left"].set_color("gray")
plt.gca().spines["bottom"].set_color("gray")
plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.legend(fontsize=8)

if os.path.exists("../files/images") is False:
    os.makedirs("../files/images")

plt.savefig("../files/images/precios.png", bbox_inches="tight")

plt.show()